In [10]:
import matplotlib.pyplot as plt
import numpy
import numpy as np
import pandas
import pandas as pd
import seaborn as sns
import re

import tensorflow as tf
from pandas import DataFrame

import nbimporter

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)
pd.set_option('display.max_columns', None)

In [11]:
def get_biosignal_in_range(biosignals, first_range, second_range):
    return biosignals.loc[(first_range+2 <= biosignals['Czas']) & (biosignals['Czas'] < second_range+2)]

In [12]:
def read_biosignal_data(biosignal_path):
    biosignals = pd.read_csv(biosignal_path, sep=',', error_bad_lines=False, index_col=False, dtype='unicode').iloc[1:,:11]
    for columnName in biosignals.columns.values:
        biosignals[columnName] = pd.to_numeric(biosignals[columnName], errors='coerce')

    return biosignals

In [13]:
def read_tetris_data(tetris_data_path):
    dataset = pd.read_csv(tetris_data_path, sep='\t', error_bad_lines=False, index_col=False, dtype='unicode')
    dataset['time'] = pd.to_numeric(dataset['time'], errors='coerce')
    dataset['player'] = pd.to_numeric(dataset['player'], errors='coerce')
    
    return dataset

In [14]:
LINES_PATTERN = "(?<=lines\()[^)]*(?=\))"

def get_first_player_tetris_state(data):
    max_time = data.tail(1)['time'].item()
    tetris_result = []
    first_player = 1
    first_player_lines = 0
    second_player_lines = 0
    i = 0

    while i < max_time:
        temp_data = data.loc[data['time']==i]

        if temp_data.empty:
            method_name(first_player_lines,second_player_lines,tetris_result)
            i+=1
            continue

        for index, row in temp_data.iterrows():
            if re.search(LINES_PATTERN, row['move']):
                line_number = int(re.findall(LINES_PATTERN, row['move'])[0])
                if row['player'] == first_player:
                    first_player_lines = line_number
                else:
                    second_player_lines = line_number
                    
        method_name(first_player_lines,second_player_lines,tetris_result)
        i+=1

    return tetris_result

def method_name(first_player_lines, second_player_lines, tetris_result):
    if first_player_lines > second_player_lines:
        tetris_result.append(1)
    elif first_player_lines < second_player_lines:
        tetris_result.append(-1)
    else:
        tetris_result.append(0)

In [15]:
def get_second_player_tetris_state(data):
    first_player = get_first_player_tetris_state(data)

    return [state * -1 for state in first_player]

In [16]:
def merge_biosignals_with_tetris_results(biosignals, tetris_results, time_start):
    tetris_max_time = len(tetris_results)
    bio_start_time = biosignals.head(1)['Czas'].item()
    bio_end_time = biosignals.tail(1)['Czas'].item()

    if 0 > (tetris_max_time - (bio_end_time - bio_start_time)) or (tetris_max_time - (bio_end_time - bio_start_time)) > 1:
        raise ValueError('Tetris times are not compatible with biosignals times')

    biosignals_with_inserted_tetris_result = insert_tetris_result(biosignals, tetris_results, time_start)

    return pd.get_dummies(biosignals_with_inserted_tetris_result, prefix='', prefix_sep='')

def insert_tetris_result(biosignals, tetris_result, time_start):
    merged_tetris_result_with_biosignals = biosignals.apply(lambda x: tetris_result[int(x['Czas'])-2-time_start], axis=1)
    biosignals = biosignals.assign(tetris=merged_tetris_result_with_biosignals.values)
    biosignals['tetris'] = biosignals['tetris'].map({-1: 'Lose', 0: 'Draw', 1: 'Win'})

    return biosignals

In [17]:
def get_players_results(first_measurement_start_time, first_measurement_end_time, second_measurement_start_time, second_measurement_end_time, folder_name):

    biosignals = read_biosignal_data(folder_name + '/biosignals.txt')
    first_round_ranged_biosignals = get_biosignal_in_range(biosignals, first_measurement_start_time, first_measurement_end_time)
    second_round_ranged_biosignals = get_biosignal_in_range(biosignals, second_measurement_start_time, second_measurement_end_time)

    first_round_tetris_data = read_tetris_data(folder_name + '/tetris_first_round.txt')
    second_round_tetris_data = read_tetris_data(folder_name + '/tetris_second_round.txt')

    first_player_tetris_state_first_round = get_first_player_tetris_state(first_round_tetris_data)
    second_player_tetris_state_first_round = get_second_player_tetris_state(first_round_tetris_data)
    first_player_tetris_state_second_round = get_first_player_tetris_state(second_round_tetris_data)
    second_player_tetris_state_second_round = get_second_player_tetris_state(second_round_tetris_data)


    first_player_first_round = merge_biosignals_with_tetris_results(first_round_ranged_biosignals, first_player_tetris_state_first_round, first_measurement_start_time)
    second_player_first_round = merge_biosignals_with_tetris_results(first_round_ranged_biosignals, second_player_tetris_state_first_round, first_measurement_start_time)

    first_player_second_round = merge_biosignals_with_tetris_results(second_round_ranged_biosignals, first_player_tetris_state_second_round, second_measurement_start_time)
    second_player_second_round = merge_biosignals_with_tetris_results(second_round_ranged_biosignals, second_player_tetris_state_second_round, second_measurement_start_time)

    return {
        'p1r1': first_player_first_round,
        'p2r1': second_player_first_round,
        'p1r2': first_player_second_round,
        'p2r2': second_player_second_round,
    }


In [18]:
dupa11 = get_players_results(0, 62, 78, 137, 's01')